In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import syllables
import re
import stanza
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [3]:
pos = pd.read_csv('../Data/POS_Columns.csv',)

In [4]:
df = pd.read_csv('../Data/train.csv', encoding = 'utf-8', lineterminator="\n")

In [5]:
nlp = stanza.Pipeline('en', logging_level='WARN', processors='tokenize,pos,constituency, sentiment')

In [6]:
text = df['excerpt']

In [7]:
def convert_string(excerpt):
    string = ''

    for i in excerpt:
        i = i.replace("\n", "")
        string += i
    return string

In [8]:
test = convert_string(text[1])

In [9]:
# doc = nlp(test)

In [10]:

# def treeWalk(child):
    
#         if len(child) == 0:
#             return 1
        
#         depth = 0
        
#         for i in range(len(child)):
#             depth = depth + treeWalk(child[i].children)
#         return depth

# i = 0
# for sent in doc.sentences:
#     tree = sent.constituency
#     print(tree)
#     child = tree.children
#     print(treeWalk(child))
        

In [11]:
# results = []
# count = 0
# for e in text:
#     doc = nlp(e)
#     depths = []
#     for sent in doc.sentences:
#         tree = sent.constituency
#         child = tree.children
#         depths.append(treeWalk(child))
#     results.append([max(depths),round(sum(depths)/len(depths), 2)])
#     print(count)
#     count += 1


In [13]:
# result_Array = np.array(results)
# pd.DataFrame(result_Array).to_csv('../Data/ParseTree.csv')
cols = ['Index','MaxDepth', 'AVGDepth']
dfTree = pd.read_csv('../Data/ParseTree.csv')
dfTree.columns = cols
dfTree = dfTree.drop(['Index'], axis = 1)

In [14]:
dfPOS = pd.read_csv('../Data/POS_Columns.csv', encoding = 'utf-8', lineterminator="\n")
cols = ['Index', 'Sents', 'Words', 'C_W_C', 'S_C',
                'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET','INTJ', 'NOUN', 
                'NUM', 'PART', 'PRON','PROPN', 'PUNCT','SCONJ','SYM','VERB','X']
dfPOS.columns = cols
dfPOS = dfPOS.drop(['X'], axis = 1)

In [15]:
dfTrain = pd.read_csv('../Data/train.csv', encoding = 'utf-8', lineterminator="\n")
target = dfTrain['target'].to_frame()
target.columns = ['Values']

In [30]:
dfAll = dfPOS.copy()
dfAll['MaxDepth'] = dfTree['MaxDepth']
dfAll['AVGDepth'] = dfTree['AVGDepth']

In [31]:
dfAll.describe()

,Index,Sents,Words,C_W_C,S_C,ADJ,ADP,ADV,AUX,CCONJ,...,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,MaxDepth,AVGDepth
count,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,...,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.000000,2834.00000,2834.000000,2834.000000
mean,1416.500000,9.181369,174.905787,19.942484,263.889908,13.689838,20.860268,8.975653,10.708186,7.711715,...,2.159845,4.277347,16.291814,6.604799,26.197248,4.469654,0.100565,22.47777,45.381793,25.297368
std,818.249656,4.308226,17.287358,11.818572,39.764395,5.485030,5.663735,4.172563,4.147206,3.302785,...,2.849353,2.632309,8.520934,6.766056,9.077591,2.620340,0.510510,6.21588,18.033296,10.369027
min,0.000000,2.000000,135.000000,0.000000,159.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,6.00000,10.000000,6.100000
25%,708.250000,6.000000,161.000000,11.000000,235.000000,10.000000,17.000000,6.000000,8.000000,5.000000,...,0.000000,2.000000,9.000000,1.000000,20.000000,2.000000,0.000000,18.00000,34.000000,18.730000
50%,1416.500000,8.000000,177.000000,18.000000,261.000000,13.000000,21.000000,9.000000,10.000000,7.000000,...,1.000000,4.000000,16.000000,5.000000,24.000000,4.000000,0.000000,22.00000,42.000000,24.140000
75%,2124.750000,11.000000,190.000000,27.000000,292.000000,17.000000,25.000000,12.000000,13.000000,10.000000,...,3.000000,6.000000,22.000000,10.000000,30.000000,6.000000,0.000000,26.00000,54.000000,30.000000
max,2833.000000,39.000000,208.000000,68.000000,409.000000,42.000000,50.000000,29.000000,29.000000,28.000000,...,28.000000,17.000000,45.000000,54.000000,77.000000,21.000000,9.000000,45.00000,175.000000,113.500000


In [33]:
sdg_reg = SGDRegressor()
scaler = StandardScaler()
scaledMetric = scaler.fit_transform(np.array(dfAll))
sdg_reg.fit(scaledMetric, target["Values"].values)
sdg_scores = cross_val_score(sdg_reg, scaledMetric, target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
sdg_scores = - sdg_scores
sdg_scores, sdg_scores.mean(), sdg_scores.std()

(array([0.63547564, 0.67556796, 0.70234295, 0.69366674, 0.63662109,
        0.63291814, 0.67115844, 0.65374728, 0.53785325, 0.64725602]),
 0.6486607504727384,
 0.04360073375867864)

In [34]:
tree_reg = DecisionTreeRegressor()
scores = cross_val_score(tree_reg, dfAll,target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
scores = - scores
scores, scores.mean(), scores.std()

(array([0.98010557, 0.87457619, 0.84912866, 0.87538642, 0.83627145,
        0.8210236 , 0.77706706, 0.92465451, 0.83856826, 0.95565465]),
 0.8732436384473724,
 0.059990177780669805)

In [36]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(scaledMetric, target["Values"].values)
gb_scores = cross_val_score(gb_reg, scaledMetric, target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
gb_scores = - gb_scores
gb_scores, gb_scores.mean(), gb_scores.std()

(array([0.67697168, 0.60939121, 0.58564589, 0.65396372, 0.56042341,
        0.56107372, 0.6043913 , 0.6550868 , 0.53167059, 0.66137828]),
 0.6099996603152341,
 0.047647276175508485)

In [38]:
rf_reg = RandomForestRegressor()
rf_reg.fit(scaledMetric, target["Values"].values)
rf_scores = cross_val_score(rf_reg, scaledMetric, target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
rf_scores = - rf_scores
rf_scores, rf_scores.mean(), rf_scores.std()

(array([0.7077319 , 0.62788386, 0.59588917, 0.64508332, 0.56390411,
        0.57624387, 0.57231943, 0.6660265 , 0.53686705, 0.69570669]),
 0.6187655887205901,
 0.05572263565829568)

In [39]:
dfAll.to_csv('../Data/AllHeuristics.csv')